In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:

from transformers import AutoModel, AutoTokenizer
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
model_name = "csebuetnlp/mT5_m2m_crossSum"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
get_lang_id = lambda lang: tokenizer._convert_token_to_id(model.config.task_specific_params["langid_map"][lang][1])



def generate(article_text,target_lang):
  input_ids = tokenizer(
      [WHITESPACE_HANDLER(article_text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]
  output_ids = model.generate(
      input_ids=input_ids,
      decoder_start_token_id=get_lang_id(target_lang),
      max_length=84,
      no_repeat_ngram_size=2,
      num_beams=4,
  )[0]
  return output_ids


path="/content/drive/My Drive/0Fyp/summ/news_data_200.csv"
#path='news_data_10.csv'
df = pd.read_csv(path)
print(df.head())


n=len(df)


target_lang = "tamil" #give target language
given_id=[]
generated_summary=[]
for i in range(n):
  article_text=df.iloc[i]['article']
  given_id.append(df.iloc[i]['id'])
  output_ids=generate(article_text,target_lang)
  summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
  )
  generated_summary.append(summary)
  print(i)

k = pd.DataFrame({'id':given_id,'summary':generated_summary})

# saving the dataframe
k.to_csv('summ_m2m_200.csv')



   id                                            article
0   1  நாலு ஆள் உயரம், முறுக்கு மீசை, கையில் வீச்சரிவ...
1   2  அமானுஷ்யமான சம்பவங்கள் நம்மை சுற்றி ஆங்காங்கே ...
2   3  காமன்வெல்த் போட்டி ஏற்பாட்டில் நடைபெற்ற முறைகே...
3   4  தென்அமெரிக்க நாடான பெருவில் காடுகள் மிகவும் பய...
4   5  கடந்த 18ம் தேதி சாயங்காலம்... அடைமழையை கிழித்த...
0
1
2
3
4
5
6
7
8
9


In [ ]:
import shutil
shutil.copy('summary.csv', '/content/drive/My Drive/0Fyp/summ')

'/content/drive/My Drive/0Fyp/summ/summary.csv'